In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
cd '/content/drive/MyDrive/Data Processing/Data for database/Utah/utah_output'

/content/drive/.shortcut-targets-by-id/1pBbvU65l2gp7KNk7HfHrTBVuRp0GUffJ/Data Processing/Data for database/Utah/utah_output


In [4]:
!pip install turfpy
!pip install mysql.connector
!pip install geopandas
!pip install tqdm

In [5]:
from turfpy import measurement
from geojson import Point, Feature
from turfpy.transformation import intersect
from turfpy.measurement import length
import json
import mysql.connector
import geopandas as gpd
from tqdm import tqdm

In [20]:
"""
29_175 : 04
29_179 : 05
36_143 : 06
"""
threads = [36]
rounds = [143]
dp_number = ['06']
state_number = '49'

In [22]:
for thread, round, districting_plan_number in zip(threads, rounds, dp_number):

  # remapping census blocks
  recomPrecincts = json.load(open(f'recom_{thread}_{round}.json'))
  # create new district mapping from ID to new District
  recomPrecinctMapping = {} # key: ID, value = District
  for node in recomPrecincts['nodes']:
    recomPrecinctMapping[int(node['id'])] = node['district']
  censusBlocks = json.load(open('../utahPreprocessedCensusBlocks.json'))
  oldDistricts = json.load(open('../utahGraph2.json'))
  # 2. generate GEOID to district mapping
  newMapping = {}
  for node in oldDistricts['nodes']:
    newMapping[node['GEOID20']] = recomPrecinctMapping[node['id']]
  # update cB files 
  for cB in censusBlocks['features']:
    cB['properties']['CD116'] = newMapping[cB['properties']['PRECINCTID']]
  # dump to JSON (1 min 52 seconds)
  with open(f'census block output/utahRecomCB_{thread}_{round}.json', 'w') as f:
    json.dump(censusBlocks, f, ensure_ascii=False)
  df = gpd.read_file(f'census block output/utahRecomCB_{thread}_{round}.json')
  df2 = df.dissolve(by='CD116', aggfunc='sum')
  df2.to_file(f'congressional district input/utahPostprocessedCongressionalDistricts_{districting_plan_number}.json', driver='GeoJSON')
  
  # Writing District Adjacencies for Border Census Blocks
  districts = json.load(open(f'congressional district input/utahPostprocessedCongressionalDistricts_{districting_plan_number}.json')) # write in the redistricting number
  def returnFeature(cB):
    return Feature(geometry={'coordinates': cB['geometry']['coordinates'], 'type': 'Polygon'}) 
  for cB in censusBlocks['features']:
    cB['properties']['FEATURE'] = returnFeature(cB)
  for district in districts['features']:
    district['properties']['FEATURE'] = returnFeature(district)
  # Census Block Adjacency
  for cB in censusBlocks['features']:
    for district in districts['features']:
      if cB['properties']['CD116'] != district['properties']['CD116']:
        if len(cB['geometry']['coordinates'][0]) >= 3 and len(district['geometry']['coordinates'][0]) >= 3:
          it = intersect([cB['properties']['FEATURE'], district['properties']['FEATURE']])
          if it:
            boundaryLen = length(it, units="ft")
            if boundaryLen >= 10:
              cB['properties']['ADJACENT_CD'] = district['properties']['CD116']
              break
  # deleting FEATURE property to reduce bloat
  for cB in censusBlocks['features']:
    del cB['properties']['FEATURE']
  for district in districts['features']:
    del district['properties']['FEATURE']
  
  # SQL Code
  mydb = mysql.connector.connect(
    host='mysql3.cs.stonybrook.edu',
    user='MothBalls',
    password='changeit',
    database='MothBalls',
    connection_timeout=3600
  )
  mycursor = mydb.cursor()
  population_sql = "INSERT INTO population (\
  geoid20, \
  cvap_american_indian_population, \
  cvap_asian_population, \
  cvap_black_population, \
  cvap_hawaiian_population, \
  cvap_hispanic_population, \
  cvap_other_population, \
  cvap_total_population, \
  cvap_white_population, \
  democrat_voters, \
  other_voters, \
  republican_voters, \
  total_american_indian_population, \
  total_asian_population, \
  total_black_population, \
  total_hawaiian_population, \
  total_hispanic_population, \
  total_other_population, \
  total_total_population, \
  total_white_population, \
  vap_american_indian_population, \
  vap_asian_population, \
  vap_black_population, \
  vap_hawaiian_population, \
  vap_hispanic_population, \
  vap_other_population, \
  vap_total_population, \
  vap_white_population\
  ) VALUES (%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)"

  censusblock_sql = "INSERT INTO census_block (\
  geoid20, \
  adjacent_congressional_district, \
  congressional_district, \
  congressional_districtid, \
  districting_plan, \
  precinctid, \
  state, \
  geometry_type, \
  geometry \
  ) VALUES (%s,%s,%s,%s,%s,%s,%s,%s,%s)"

  district_sql = "INSERT INTO district (\
  geoid20, \
  congressional_district, \
  districting_plan, \
  state \
  ) VALUES (%s,%s,%s,%s)"

  districting_plan_sql = "INSERT INTO districting_plan (\
  id, \
  districting_plan, \
  state \
  ) VALUES (%s,%s,%s)"
  # INSERT INTO DISTRICTING_PLAN TABLE
  val = ((districting_plan_number + state_number), \
          districting_plan_number, \
          state_number \
  )
  mycursor.execute(districting_plan_sql, val)
  # INSERT INTO DISTRICT TABLE
  for feature in districts['features']:
    properties = feature['properties']
    val = ((districting_plan_number + state_number + properties['CD116']), \
          properties['CVAP_AMERICANINDIAN'], \
          properties['CVAP_ASIAN'], \
          properties['CVAP_BLACK'], \
          properties['CVAP_HAIWAIIAN'], \
          properties['CVAP_HISPANIC'], \
          properties['CVAP_OTHER'], \
          properties['CVAP_TOTAL'], \
          properties['CVAP_WHITE'], \
          properties['DEMOCRAT'], \
          properties['OTHER'], \
          properties['REPUBLICAN'], \
          properties['TOTAL_AMERICANINDIAN'], \
          properties['TOTAL_ASIAN'], \
          properties['TOTAL_BLACK'], \
          properties['TOTAL_HAWAIIAN'], \
          properties['TOTAL_HISPANIC'], \
          properties['TOTAL_OTHER'], \
          properties['TOTAL_TOTAL'], \
          properties['TOTAL_WHITE'], \
          properties['VAP_AMERICANINDIAN'], \
          properties['VAP_ASIAN'], \
          properties['VAP_BLACK'], \
          properties['VAP_HAWAIIAN'], \
          properties['VAP_HISPANIC'], \
          properties['VAP_OTHER'], \
          properties['VAP_TOTAL'], \
          properties['VAP_WHITE'] \
    )
    mycursor.execute(population_sql, val)

    val = ((districting_plan_number + state_number + properties['CD116']), \
          properties['CD116'], \
          districting_plan_number, \
          state_number \
    )
    mycursor.execute(district_sql, val)

  # INPUT DATA INTO POPULATION TABLE AND CENSUS BLOCK TABLE FOR ALL CENSUS BLOCKS
  print('reached population insert')

  pop_vals = []
  cb_vals = []
  for feature in censusBlocks['features']:
    properties = feature['properties']
    pop_val = ((districting_plan_number + properties['GEOID20']), \
          properties['CVAP_AMERICANINDIAN'], \
          properties['CVAP_ASIAN'], \
          properties['CVAP_BLACK'], \
          properties['CVAP_HAIWAIIAN'], \
          properties['CVAP_HISPANIC'], \
          properties['CVAP_OTHER'], \
          properties['CVAP_TOTAL'], \
          properties['CVAP_WHITE'], \
          properties['DEMOCRAT'], \
          properties['OTHER'], \
          properties['REPUBLICAN'], \
          properties['TOTAL_AMERICANINDIAN'], \
          properties['TOTAL_ASIAN'], \
          properties['TOTAL_BLACK'], \
          properties['TOTAL_HAWAIIAN'], \
          properties['TOTAL_HISPANIC'], \
          properties['TOTAL_OTHER'], \
          properties['TOTAL_TOTAL'], \
          properties['TOTAL_WHITE'], \
          properties['VAP_AMERICANINDIAN'], \
          properties['VAP_ASIAN'], \
          properties['VAP_BLACK'], \
          properties['VAP_HAWAIIAN'], \
          properties['VAP_HISPANIC'], \
          properties['VAP_OTHER'], \
          properties['VAP_TOTAL'], \
          properties['VAP_WHITE'] \
    )
    pop_vals.append(pop_val)
    if 'ADJACENT_CD' in properties:
      cb_val = ((districting_plan_number + properties['GEOID20']), \
          properties['ADJACENT_CD'], \
          properties['CD116'], \
          (districting_plan_number + properties['STATE'] + properties['CD116']), \
          districting_plan_number, \
          properties['PRECINCTID'], \
          properties['STATE'], \
          feature['geometry']['type'], \
          json.dumps(feature['geometry']['coordinates']), \
      )
      cb_vals.append(cb_val)
    else:
      cb_val = ((districting_plan_number + properties['GEOID20']), \
          None, \
          properties['CD116'], \
          (districting_plan_number + properties['STATE'] + properties['CD116']), \
          districting_plan_number, \
          properties['PRECINCTID'], \
          properties['STATE'], \
          feature['geometry']['type'], \
          json.dumps(feature['geometry']['coordinates']), \
      )
      cb_vals.append(cb_val)
  # mycursor.executemany(population_sql, pop_vals)
  small_pop_vals = []
  i = 0
  for pop_val in tqdm(pop_vals):
    i += 1
    if i > 5000:
      mycursor.executemany(population_sql, small_pop_vals)
      small_pop_vals = []
      i = 0
    small_pop_vals.append(pop_val)
  mycursor.executemany(population_sql, small_pop_vals)

  mydb.ping(True)
  print('reached census block inserts')

  small_cb_vals = []
  i = 0
  for cb_val in tqdm(cb_vals):
    i += 1
    if i > 500:
      mycursor.executemany(censusblock_sql, small_cb_vals)
      small_cb_vals = []
      i = 0
    small_cb_vals.append(cb_val)
  mycursor.executemany(censusblock_sql, small_cb_vals)
  # INPUT DATA INTO DISTRICTING PLAN STATISTICS TABLE
  districting_plan_statistics_sql = "INSERT INTO districting_plan_statistics (\
  id, \
  redistrict_number, \
  state, \
  total_population_score, \
  vap_population_score, \
  cvap_population_score, \
  efficiency_gap, \
  num_congressional_districts, \
  num_democratic_congressional_districts, \
  num_republican_congressional_districts \
  ) VALUES (%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)"

  recomStatistics = json.load(open(f'recom_statistics_{thread}_{round}.json'))

  #MAKE SURE TO CHANGE NUMBERS FOR UTAH
  val = ((districting_plan_number + state_number), \
          districting_plan_number, \
          state_number, \
          recomStatistics['total_pop_score_diff'], \
          recomStatistics['vap_pop_score_diff'], \
          recomStatistics['cvap_pop_score_diff'], \
          recomStatistics['eg'], \
          4, \
          recomStatistics['d_districts'], \
          recomStatistics['r_districts'] \
  )
  mycursor.execute(districting_plan_statistics_sql, val)


  mydb.commit()
  mydb.close()

  print(f'{thread}_{round}: {districting_plan_number} complete')


reached population insert


100%|██████████| 71207/71207 [00:17<00:00, 4154.77it/s]


reached census block inserts


100%|██████████| 71207/71207 [02:05<00:00, 566.57it/s]


36_143: 06 complete
